# Prokaryotic Promoters

This working prototype that reads a single string, verifies that it is a valid DNA sequence, finds exact matches to common promoter motifs, and reports an overall match score for each proposed promoter.

###  The DNA verification process

DNA normally consists of 4 different nucleotieds **A, T, G, and C.** We can scan the string to check if we find any string other than these characters. Linear Scanning can perform this operation in **O(n)** time.

### Promoter Matching Process

We can locate promoters using an exact matching algorithm. This process is very straightforward however is not always accurate. Fuzzy matching process can locate multiple possible promoter regions. Fuzzy matching technique generates a list of possible promoter regions with their corresponding probability of being a promoter.

##### Exact Matching

For exact promoter matching, Two regions indicating a promoter found together are the  **TTGACA (-35) and the TATAAT (-10)** regions. Which means, we can simply scan for these two regions to locate the promoter start regions. 

We can keep track of how spread apart **TTGACA(-35) and TATAAT (-10)** are, making sure their **distance (d)** $\le$ 21 and $\ge$ 15

To make things simple, for exact matching we will be looking for **TTGACA** and **TATAAT** or **TAXXXT** blocks.

To calculate the overall confidence score:
    
- We can use the normal distribution formula constrained between the range [21,15] where $\mu = 17$, and $\sigma = 0.8$.
- We calculate the (-35) or (-10) block probability using the corresponding hashtable (Dan's Research) for each block type we have. 

In [10]:
# Hashtable for -35 sequence
tf_hashtable = {
    "A" : [3,  10, 3, 58, 32, 54],
    "C" : [9,  3, 14, 13, 52,  5],
    "G" : [10, 5, 68, 10,  7, 17],
    "T" : [78, 82,15, 20, 10, 24],
}

# Hashtable for -10 sequence
t_hashtable = {
    "A" : [3, 89, 26, 59, 49, 3],
    "C" : [8, 3,  10, 12, 21, 5],
    "G" : [7, 1,  12, 15, 11, 2],
    "T" : [82, 7, 52, 14, 19, 89],
}

### Exact matching algorithm with hash table

If we consider a nucleotide $\tau$ at position $n$ in a DNA sequence $\Delta$ as the start of the -35 sequence

Given an example DNA Sequence ...TAGACAC...

Considering $\tau=$ **T** with position $n$ then we can generate a scoring system like below using the `tf_hashtable`:
| T | A | G | A | C | A | $\mu$ |
|---|---|---|---|---|---|-------|
|78 | 10| 68| 20|52 | 54| $\frac{47}{100}$   |

where $\mu$ is the score of the possible -35 sequence.



Once this step is done, we move forward i.e we go to $n + 1$ location where $\tau=$ A where we get another score for the nucleotide subset:

| A | G | A | C | A | C | $\mu$ |
|---|---|---|---|---|---|-------|
| 10| 68| 20| 52| 54| 5 | $\frac{-26}{100}$   |

Ideal way of calculating $\mu$ after the first subset is to subtract score at $n$ and add the score at $n + 1 + 5$ 

We store the list key=n value=[-35 sequence, score] ($\Beta$) as a list. For the **-10** sequence we create a dictionary with [-10 sequence, n, score] ($\Alpha$).


In [ ]:
DNA_Sequence = "ATGCGTACGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC"


In [16]:
# Beta
tf_dict = {}

# Alpha
t_list = []

# Final List

flist = []

# calculating -35 scores
for n, T in enumerate(DNA_Sequence):
    if n + 6 <= len(DNA_Sequence):
        score = 0
        for j in range(0, 6):
            score += tf_hashtable[DNA_Sequence[n]][j]
        mu = (score) / 600
        tf_dict[n] = [DNA_Sequence[n:n + 6], mu]

In [9]:
tf_dict

{0: ['ATGCGT', 0.26666666666666666],
 1: ['TGCGTA', 0.38166666666666665],
 2: ['GCGTAC', 0.195],
 3: ['CGTACG', 0.16],
 4: ['GTACGT', 0.195],
 5: ['TACGTA', 0.38166666666666665],
 6: ['ACGTAG', 0.26666666666666666],
 7: ['CGTAGC', 0.16],
 8: ['GTAGCT', 0.195],
 9: ['TAGCTA', 0.38166666666666665],
 10: ['AGCTAG', 0.26666666666666666],
 11: ['GCTAGC', 0.195],
 12: ['CTAGCT', 0.16],
 13: ['TAGCTA', 0.38166666666666665],
 14: ['AGCTAG', 0.26666666666666666],
 15: ['GCTAGC', 0.195],
 16: ['CTAGCT', 0.16],
 17: ['TAGCTA', 0.38166666666666665],
 18: ['AGCTAG', 0.26666666666666666],
 19: ['GCTAGC', 0.195],
 20: ['CTAGCT', 0.16],
 21: ['TAGCTA', 0.38166666666666665],
 22: ['AGCTAG', 0.26666666666666666],
 23: ['GCTAGC', 0.195],
 24: ['CTAGCT', 0.16],
 25: ['TAGCTA', 0.38166666666666665],
 26: ['AGCTAG', 0.26666666666666666],
 27: ['GCTAGC', 0.195],
 28: ['CTAGCT', 0.16],
 29: ['TAGCTA', 0.38166666666666665],
 30: ['AGCTAG', 0.26666666666666666],
 31: ['GCTAGC', 0.195],
 32: ['CTAGCT', 0.16],
 3

In [11]:
# calculating -10 scores
for n, T in enumerate(DNA_Sequence):
    if n + 6 <= len(DNA_Sequence):
        score = 0
        for j in range(0, 6):
            score += t_hashtable[DNA_Sequence[n]][j]
        mu = (score) / 600
        t_list.append([DNA_Sequence[n:n + 6], n, mu])

In [12]:
t_list

[['ATGCGT', 0, 0.38166666666666665],
 ['TGCGTA', 1, 0.43833333333333335],
 ['GCGTAC', 2, 0.08],
 ['CGTACG', 3, 0.09833333333333333],
 ['GTACGT', 4, 0.08],
 ['TACGTA', 5, 0.43833333333333335],
 ['ACGTAG', 6, 0.38166666666666665],
 ['CGTAGC', 7, 0.09833333333333333],
 ['GTAGCT', 8, 0.08],
 ['TAGCTA', 9, 0.43833333333333335],
 ['AGCTAG', 10, 0.38166666666666665],
 ['GCTAGC', 11, 0.08],
 ['CTAGCT', 12, 0.09833333333333333],
 ['TAGCTA', 13, 0.43833333333333335],
 ['AGCTAG', 14, 0.38166666666666665],
 ['GCTAGC', 15, 0.08],
 ['CTAGCT', 16, 0.09833333333333333],
 ['TAGCTA', 17, 0.43833333333333335],
 ['AGCTAG', 18, 0.38166666666666665],
 ['GCTAGC', 19, 0.08],
 ['CTAGCT', 20, 0.09833333333333333],
 ['TAGCTA', 21, 0.43833333333333335],
 ['AGCTAG', 22, 0.38166666666666665],
 ['GCTAGC', 23, 0.08],
 ['CTAGCT', 24, 0.09833333333333333],
 ['TAGCTA', 25, 0.43833333333333335],
 ['AGCTAG', 26, 0.38166666666666665],
 ['GCTAGC', 27, 0.08],
 ['CTAGCT', 28, 0.09833333333333333],
 ['TAGCTA', 29, 0.4383333333


**-10 and -35 distance**

Since the space between the promoter blocks have an effect in the overall probability. Once we get calculate $\Alpha$ and $\Beta$, we iterate through $\Beta$ 
in the following manner

for each element at index $i$ with value $\alpha \in \Alpha$:
    
where key $\gt \beta[1] - 12$ and $\lt \beta[1] - 22$ if it exists in $\Alpha$ 

if it does: we calculate final score as $S = \frac{((score_\alpha + score_\beta) + std_{-}normal(i - \Alpha[key]))}{3} $ 

append the resulting pair with the score and sort the result based on the final score




In [14]:
E = 2.718281828459045
PI = 3.141592653589793

def std_normal(mu, x, sigma):
    return 1/(sigma * (2 * PI)**0.5) * E**(-0.5 * ((x - mu)/sigma)**2)

In [19]:
# iterating through -10 sequence list

for i, n in enumerate(t_list):
    b_start = n[1] - 22
    b_end = n[1] - 12
    for k in range(b_start, b_end):
        if k in tf_dict.keys():
            final_score = ((n[2] + tf_dict[k][1]) + std_normal(17, n[1], 0.8)) / 3
            print(final_score)
            flist.append([n[0], tf_dict[k][0], n[1], k, final_score])
            # output format -10, -35, -10 index, -35 index, final score


In [18]:
flist.sort(key=lambda x: x[4], reverse=True)
flist

[]


##### Fuzzy Matching

These are the conditions to keep in mind: 
- In the TATAAT region, TAXXXT are conserved over 80% of the time (X means less conserved)

- In the TTGACA region of the first 3 (TTG) at least one is always conserved but 2 and most often 3 are usually conserved.

- Both units tend to occcur about 17 basepairs away from eachother, but this can vary from as little as 15 to as high as 21 (however 95% of the time it is betwren 16 and 18, with more than half of that being 17)

- The -35 region is further upstream from the -10 (-35 and -10 describe the base pair distance from the starting region)